In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, lorentian_distr, get_trajs, speed_windowed, theta_windowed, \
                    get_smooth_trajs, get_velocities
                   
show_verb = True
save_verb = True
anim_show_verb = False

In [53]:
#rawTrajs = pd.read_parquet("../tracking/results/tracking_data/trackpy_pre_merge.parquet")
rawTrajs = pd.read_parquet("../tracking/results/tracking_data/tracking_hough_trackpy_linking.parquet")
rawTrajs = rawTrajs.loc[rawTrajs.frame < 15000]

res_path = "hough_results"
analysis_data_path = "hough_analysis_data"
red_particle_idx = 17
rawTrajs.loc[rawTrajs.particle != red_particle_idx, ["color"]] = "#00007F"
rawTrajs.loc[rawTrajs.particle == red_particle_idx, ["color"]] = "#FF0000"
colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
nFrames = int(max(rawTrajs.frame) + 1)
print(f"nDrops:{nDrops}")
print(f"nFrames:{nFrames} --> {nFrames/10:.2f} s")


# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print(f"window of {window/10} s, stride of {stride/10} s")
startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"number of steps: {nSteps}")


# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

nDrops:50
nFrames:15000 --> 1500.00 s
window of 320.0 s, stride of 10.0 s
number of steps: 118


In [18]:
# Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

# MSD

In [20]:
show_verb = False
run_windowed_analysis = False
animated_plot_verb = False

print(f"MSD Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")

%run ./analysis_modules/msd.ipynb

MSD Analysis: show_verb = False, run_windowed_analysis = True, animated_plot_verb = True
Global IMSD
Global EMSD
Raw trajs - Blue Particles: 1.029 ± 0.001, Red Particle: 0.756 ± 0.001
Smooth trajs - Blue Particles: 1.032 ± 0.001, Red Particle: 0.759 ± 0.001
Windowed IMSD


100%|██████████| 118/118 [03:11<00:00,  1.62s/it]
14:30:19 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:30:19 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/mean_squared_displacement/windowed_analysis/IMSD_wind_raw.mp4
14:30:38 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:30:38 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/mean_squared_displacement/windowed_analysis/IMSD_wind_smooth.mp4


Windowed EMSD


/Users/matteoscandola/Desktop/MasterThesis/analysis/utility.py:25: RuntimeWarning: overflow encountered in power
  return a*x**k
14:31:00 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:31:00 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/mean_squared_displacement/windowed_analysis/EMSD_wind_raw.mp4
14:31:14 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:31:14 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/mean_squared_displacement/windowed_analysis/EMSD_wind_smooth.mp4


# VELOCITY DISTRIBUTION

In [22]:
show_verb = False
animated_plot_verb = False
print(f"Speed and Turning Angles Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_distr.ipynb

Speed and Turning Angles Analysis: show_verb = False, animated_plot_verb = True

 Global speed distribution analysis
Raw Trajectories
Blue Particles σ: 3.556602168281856 ± 0.08006041631076558
Red Particle σ: 13.616863727850623 ± 0.30158038382693375
Smooth Trajectories
Blue Particles σ: 3.2963774450296763 ± 0.011639571850313609
Red Particle σ: 13.609249100429722 ± 0.22940153907073435

 Windowed speed distribution Analysis


100%|██████████| 118/118 [00:25<00:00,  4.72it/s]
14:33:41 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:33:41 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./results/speed_distribution/speed_raw_wind.mp4
14:34:37 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:34:37 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./results/speed_distribution/speed_smooth_wind.mp4


# TURNING ANGLES DISTRIBUTIONS

In [23]:
if 0:
    distribution = normal_distr
    distribution_str = "gaussian"
else:
    distribution = lorentian_distr
    distribution_str = "lorentian"

show_verb = False
animated_plot_verb = False
print(f"Speed and Turning Angles Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/turning_angles.ipynb

Speed and Turning Angles Analysis: show_verb = False, animated_plot_verb = True

 Global turning angles analysis
Raw Trajectories
Blue Particles σ: 0.062122598774449846 ± 0.006920747038848485, μ: -0.0314344186441415 ± 0.009603820233429745
Red Particle σ: 0.06986956425489817 ± 0.006689448673887414, μ: -0.03209908275230889 ± 0.008261888384094685
Smooth Trajectories
Blue Particles σ: 0.11738271933534945 ± 0.0013108159536947992, μ: 0.0009840868669791062 ± 0.0012996054239290086
Red Particle σ: 0.08136441121501183 ± 0.0014102951366595923, μ: 0.0026559787372172168 ± 0.0012734215930431069

 Windowed turning angles analysis


100%|██████████| 118/118 [00:17<00:00,  6.89it/s]
14:36:13 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:36:13 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./results/turning_angles/lorentian/turn_ang_wind_raw.mp4
14:36:27 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:36:27 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./results/turning_angles/lorentian/turn_ang_wind_smooth.mp4


# VELOCITY AUTOCORRELATION

In [ ]:
show_verb = False
run_windowed_analysis = False
overwrite = False
animated_plot_verb = False

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

# RADIAL DISTRIBUTON FUNCTION

In [54]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

print("Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}
RDF - Raw Trajectories
raw


100%|██████████| 15000/15000 [47:53<00:00,  5.22it/s]  


RDF - Smooth Trajectories
smooth


100%|██████████| 15000/15000 [43:47<00:00,  5.71it/s]
14:37:18 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:37:18 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1000x400 -pix_fmt rgba -r 120 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/radial_distribution_function/rdf_raw.mp4
14:38:01 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:38:01 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1000x400 -pix_fmt rgba -r 120 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/radial_distribution_function/rdf_smooth.mp4


RDF from center - Raw Trajectories


100%|██████████| 15000/15000 [01:52<00:00, 132.78it/s]


RDF from center - Smooth Trajectories


100%|██████████| 15000/15000 [01:49<00:00, 137.39it/s]
14:42:34 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:42:34 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 120 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/radial_distribution_function/rdf_from_center_raw.mp4
14:43:16 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:43:16 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 120 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/radial_distribution_function/rdf_from_center_smooth.mp4


# 3 BODY STRUCTURE METRIC

In [ ]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb